In [1]:
pip install selenium


  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/e0/7a/08f0ea19a0c835e88aad011083d9dda69a9dfa4585c3453b3bd842eb7bed/selenium-4.21.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/76/51/12d78ec8abcbda51d8f115d98ebd3ee3da9d9d9af00ac69d3097c5b8d51a/trio-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa6703fcf4a79018f/attrs-23.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for sortedcontainers from https://files.pythonhosted.org/packages/32/46/9cb0e58b2deb7f82b84065f37f3bffeb12413f947f9388e4cac22c4621ce/sortedc


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import csv

# Set up the WebDriver (replace 'path_to_chromedriver' with the actual path to your chromedriver)
service = Service(r"C:\Users\Pistol Shrimp\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Navigate to the webpage
driver.get("https://engage.vic.gov.au/project/shape-our-victoria/page/housing-targets-2051")

# Wait for the page to load completely
time.sleep(5)  # Adjust the sleep time if necessary

# Find all the black dots
dots = driver.find_elements(By.CLASS_NAME, 'engage-hotspot-data-point__point')

council_data = []

# Open a CSV file to progressively save results
with open('council_housing_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Council', 'Existing Homes', 'Draft Target'])

    # Iterate over each dot and extract the data from the popup
    for dot in dots:
        try:
            # Ensure the element is clickable before clicking
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'engage-hotspot-data-point__point')))

            # Click on the dot to make the popup appear
            dot.click()
            time.sleep(2)  # Wait for the popup to appear

            # Wait until the popup appears
            popup = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'engage-hotspot-data-point__popup--container'))
            )

            # Debug print to see the raw popup text content
            print(f"Popup Text Content: {popup.text}")

            # Extract data based on inspected structure
            council_name = popup.find_element(By.TAG_NAME, 'h3').text
            content = popup.find_element(By.CLASS_NAME, 'engage-hotspot-data-point__popup--content')
            existing_homes = content.find_elements(By.TAG_NAME, 'p')[0].text.split('-')[-1].strip()
            draft_target = content.find_elements(By.TAG_NAME, 'p')[1].text.split('-')[-1].strip()

            print(f"Extracted: {council_name}, {existing_homes}, {draft_target}")

            council_data.append({
                'Council': council_name,
                'Existing Homes': int(existing_homes.replace(',', '')),
                'Draft Target': int(draft_target.replace(',', ''))
            })

            # Save to CSV
            writer.writerow([council_name, existing_homes, draft_target])

            # Close the popup
            close_button = popup.find_element(By.CLASS_NAME, 'engage-hotspot-data-point__popup--close')
            close_button.click()
            time.sleep(1)  # Wait a moment to ensure the popup is closed

        except Exception as e:
            print(f"Error extracting data: {e}")

# Close the WebDriver
driver.quit()

# Display the extracted data
df = pd.DataFrame(council_data)
print(df)


Popup Text Content: Mornington Peninsula Shire Council
2023 - number of existing homes - 94,900
2051 - draft target - 31,000
Extracted: Mornington Peninsula Shire Council, 94,900, 31,000
Popup Text Content: Frankston City Council
2023 - number of existing homes - 59,800
2051 - draft target - 36,000
Extracted: Frankston City Council, 59,800, 36,000
Popup Text Content: Greater Dandenong City Council
2023 - number of existing homes - 59,400
2051 - draft target - 57,000
Extracted: Greater Dandenong City Council, 59,400, 57,000
Popup Text Content: Knox City Council
2023 - number of existing homes - 63,100
2051 - draft target - 47,000
Extracted: Knox City Council, 63,100, 47,000
Popup Text Content: Maroondah City Council
2023 - number of existing homes - 48,000
2051 - draft target - 44,000
Extracted: Maroondah City Council, 48,000, 44,000
Popup Text Content: Yarra Ranges Shire Council
2023 - number of existing homes - 62,700
2051 - draft target - 28,000
Extracted: Yarra Ranges Shire Council,